In [9]:
import os
os.environ.get('TWILIO_AUTH_TOKEN')


'641d687ab349d2d67eb804647f1753e5'

### Twilio SMS

ACc44428eadbebe6b773ea40e9c89d8d02
641d687ab349d2d67eb804647f1753e5


In [ ]:
#!/usr/bin/env python

# libaries
import os
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from twilio.rest import Client


class CunyEnrollmentBot:
    
    def __init__(self, username, password, link, to_phone_num='+16468973317',msg=''):
        
        """
        If you donot have the Chrome driver on your machine you need to download one.
        You can download it from the link https://chromedriver.chromium.org/downloads
        """
        # path for chromedriver
        self.PATH = "/Users/tenzintashi/Downloads/chromedriver"
        
        # use Chrome as the browser
#         self.driver =  webdriver.PhantomJS()
        self.driver = webdriver.Chrome(self.PATH)

        self.link = link
        self.USERNAME = username #username
        self.PASSWORD = password #password
        self.driver.get(self.link)
        # Website to access
        # https://home.cunyfirst.cuny.edu/psp/cnyepprd/EMPLOYEE/EMPL/h/?tab=DEFAULT
        
        # For sending sms 
        self.account_sid = os.environ.get('TWILIO_ACCOUNT_SID')
        self.auth_token = os.environ.get('TWILIO_AUTH_TOKEN')
        self.from_phone_num = '+19292034870'
        self.to_phone_num = to_phone_num
        if msg == '':
            self.msg="We have added a to your schedule"
        else:
            self.msg=msg


        
    def login(self):
        # login process
        search_username = self.driver.find_element_by_id("CUNYfirstUsernameH")
        search_password = self.driver.find_element_by_id("CUNYfirstPassword")
        search_enter = self.driver.find_element_by_id("submit")

        search_username.send_keys(self.USERNAME)

        search_password.send_keys(self.PASSWORD)

        search_enter.send_keys(Keys.RETURN)

        time.sleep(2)
    
    def stop(self):
        self.driver.quit
        
    def send_sms(self):
        client = Client(self.account_sid, self.auth_token)
        client.messages.create(from_=self.from_phone_num,
                               to=self.tp_phone_num,
                               body=self.msg)
    
    def run(self):
        try:
            self.login()
            while True:
                shoppingcartlen = len(self.driver.find_elements_by_xpath("//table[@id='SSR_REGFORM_VW$scroll$0']/tbody/tr")) - 2
                shoppingcart = {}
                for i in range(shoppingcartlen):
                    classinfo = self.driver.find_element_by_id("trSSR_REGFORM_VW$0_row" + str(i+1)).text
                    classname = classinfo.split()[0]+" "+classinfo.split()[1]
                    status = self.driver.find_element_by_xpath("//div[@id='win0divDERIVED_REGFRM1_SSR_STATUS_LONG$" + str(i) + "']/div/img").get_attribute('alt')
                    shoppingcart[classname] = status

                # Enroll if a course status change to open
                if "Open" in shoppingcart.values(): 
                    # Show 'the course is open' msg in the console
                    print("Course status changed to open")

                    # Proceed to enrollment
                    proceed_btn = WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.ID, "DERIVED_REGFRM1_LINK_ADD_ENRL$82$"))
                    )
                    proceed_btn.click()

                    # Finish enrollment
                    finish_enroll_btn = WebDriverWait(self.driver, 10).until(
                        EC.presence_of_element_located((By.ID, "DERIVED_REGFRM1_SSR_PB_SUBMIT"))
                    )
                    finish_enroll_btn.click()
                    time.sleep(2)

                self.driver.get(self.link)

                if shoppingcartlen > len(self.driver.find_elements_by_xpath("//table[@id='SSR_REGFORM_VW$scroll$0']/tbody/tr")) - 2:
                    shoppingcartlen = len(self.driver.find_elements_by_xpath("//table[@id='SSR_REGFORM_VW$scroll$0']/tbody/tr")) - 2

                    # Show 'course added' msg in the console
                    print("Course successfully added !!!")
#                     self.send_sms()
                if shoppingcartlen == 0:
                    break

                time.sleep(2)
        finally:
            self.stop()
    
    

if __name__ == "__main__":
    Username = "USERNAME"
    Password =  "PASSWORD"
    Shopping_cart_link = "Link"
    tenzin_cuny = CunyEnrollmentBot(Username , Password, Shopping_cart_link)
    tenzin_cuny.run()